## Scrape historical tweets of US senators

This notebook scrapes tweets from all official accounts of US senators with `snscrape` and stores
them in a local `sqlite` database. The tweets are cleaned using the `Preprocessor` package and
modern `Pandas` chaining methods.

In [ ]:
# Imports
from datetime import datetime
import pandas as pd
import datapane as dp
import snscrape
import sqlite3
from snscrape.modules import twitter
from sqlalchemy import create_engine
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt


In [ ]:
# Read in list of senator usernames
usernames = pd.read_csv("data/senators_usernames.csv")


### Scrape and tidy tweets

In [ ]:
def extract_tweets(scraped_profile):
    """Extracts essential tweet data from a Twitter
    profile that's been previously scraped with snscrape.

    Args:
        scraped_profile (generator): Complete Twitter user profile.
    Returns:
        list: List of dictionaries each containing tweet data.
    """
    return [
        {
            "date": tweet.date.strftime("%Y-%m-%d %I:%M:%S %p"),
            "id": tweet.id,
            "username": tweet.user.username,
            "text": tweet.content,
            "is_retweet": str(tweet.retweetedTweet),
        }
        for tweet in scraped_profile
    ]


In [ ]:
def tidy_tweets(tweets):
    """Clean up raw tweet data and store in a dataframe
    with senator usernames and party affiliations.

    Args:
        tweets (list): List of dictionaries each containing tweet data.

    Returns:
        pd.DataFrame: Dataframe with all tweets for each senator as
        well as their username and party affiliation.
    """
    # Put tweet data into a df
    tweets_df = pd.DataFrame(tweets)
    # Tidy up
    return (
        tweets_df.query('is_retweet == "None"')  # remove all retweets
        .assign(date=pd.to_datetime(tweets_df.date))  # convert col from str to datetime
        # .query(f"{START_DATE} < date < {END_DATE}")  # drop tweets outside of date range
        .drop(columns=["is_retweet"])  # drop col
        .merge(usernames[["username", "party"]])  # get usernames, party affiliations
        .sort_values(by=["date"])  # sort by date
        .reset_index(drop=True)
    )


In [ ]:
# Connect to locally created db (and create if it doesn't exist)
conn = sqlite3.connect("data/TWEETS.db")  # path to db


In [ ]:
# Initalize empty list for tweets data
dfs = []

# Initialize empty list for skipped usernames
skipped_usernames = []

# Get tweets by username and append to db
for username in usernames.username.tolist():
    # for username in ["RogerMarshallMD", "LeaderMcConnell"]:
    try:
        # Get all tweets for given username
        scraped_profile = twitter.TwitterProfileScraper(username=username).get_items()
        # Extra tweet metadata
        tweets = extract_tweets(scraped_profile)
        # Tidy up tweets and put in df
        tweets_df = tidy_tweets(tweets)
        # Append to list
        dfs.append(tweets_df)
    except:
        print(f"Couldn't get tweets for {username}")
        # Get list of all skipped usernames
        skipped_usernames.append(username)


### Write tweets to SQLite

In [ ]:
# Write df to senators table in Tweets db
for df in dfs:
    df.to_sql("senators", conn, if_exists="append", chunksize=10000)


### Investigate the data

In [ ]:
# Read back results from SQL table
sql_df = pd.read_sql("select * from senators", conn)

# Select only tweets from current session of Congress in 2021
sql_df = sql_df.query(
    f"'2021-01-20' <= date <= '2021-12-31'"
)  # drop tweets outside of date range
sql_df.shape


#### Which party tweeted more?

In [ ]:
# Create a mapping for more a descriptive visualization
party_mapping = {"D": "Democrat", "R": "Republican", "I": "Independent"}

# Get number of tweets by party
pie_df = (
    sql_df.assign(party=sql_df.party.replace(party_mapping))
    .groupby("party")["text"]
    .count()
    .reset_index()
    .rename(columns={"text": "Tweets", "party": "Party"})
)

# Plot pie chart of senator tweets by party membership
fig_pie = px.pie(
    pie_df,
    values="Tweets",
    names="Party",
    color="Party",
    color_discrete_map={
        "Democrat": "RoyalBlue",
        "Republican": "FireBrick",
        "Independent": "Silver",
    },
    hover_name="Tweets",
)
fig_pie.update_layout(
    title={
        "text": "Senator tweets by party (2021)",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22},
    },
)
fig_pie.update_traces(
    hovertemplate="Tweets: %{value} <br>Party: %{label}",
    textinfo="percent+value",
    textfont_size=13,
)
fig_pie.show()


#### Who are the top tweeting senators?

In [ ]:
# Get tweets per senator
bar_df = (
    sql_df.assign(party=sql_df.party.replace(party_mapping))
    .groupby(["username", "party"])["text"]
    .count()
    .reset_index()
    .merge(usernames[["name", "username", "state"]])
    .sort_values(by="text", ascending=False)
    .rename(
        columns={
            "text": "Tweets",
            "party": "Party",
            "name": "Senators",
            "state": "State",
            "username": "Username",
        }
    )
)

# Create bar plot of the top 20 tweeting senators
fig_bar = px.bar(
    bar_df.head(20),
    x="Senators",
    y="Tweets",
    color="Party",
    color_discrete_map={
        "Democrat": "RoyalBlue",
        "Republican": "FireBrick",
        "Independent": "Silver",
    },
    height=600,
)

fig_bar.update_layout(
    title={
        "text": "Top 20 tweeting senators (2021)",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22},
    },
    xaxis=dict(titlefont_size=18),
    yaxis=dict(titlefont_size=18),
    xaxis_categoryorder="total descending",
    xaxis_tickangle=-45,
    xaxis_tickfont_size=14,
)

fig_bar.update_yaxes(range=[0, 4000])

fig_bar.add_hline(
    y=bar_df.Tweets.mean(), line_width=2, line_dash="dash", line_color="lightgray"
)

fig_bar.update_traces(
    hovertemplate="Tweets: %{y} <br>Name: %{x}",
    textfont_size=13,
)

fig_bar.show()


In [ ]:
# Save plots locally to html
fig_bar.write_html("plots/tweets-by-senator.html")
fig_pie.write_html("plots/tweets-by-party.html")

# Upload each figure to it's own datapane report
dp.Report(
    dp.Plot(fig_pie),
).upload(name="Senator tweets by party in 2021")
dp.Report(
    dp.Plot(fig_bar),
).upload(name="Top 20 tweeting senators in 2021")


#### Tweets word clouds

In [ ]:
# Set stopwords for both word clouds
stopwords = set(STOPWORDS)
stopwords.update(
    [
        "https",
        "s",
        "t",
        "co",
        "amp",
        "u",
        "today",
        "make",
        "re",
        "m",
        "will",
        "need",
        "support",
        "thank",
        "state",
        "bill",
        "nation",
        "time",
        "American",
        "now",
        "work",
        "help",
        "continue",
        "country",
        "year",
        "one",
        "new",
        "day",
        "week",
        "see",
        "live",
        "ensure",
        "many",
        "family",
        "keep",
        "made",
        "discuss",
        "sure",
        "people",
        "let",
        "must",
        "take",
        "continue",
        "effort",
        "thanks",
        "Thanks",
        "way",
        "know",
        "us",
        "many",
        "don",
        "w",
        "go",
    ]
)

# Join all strings of tweets for each party
cloud_df = sql_df.groupby("party").agg({"text": " ".join}).reset_index()


In [ ]:
# Generate word cloud image for Democratic tweets
wordcloud_dems = WordCloud(
    stopwords=stopwords,
    background_color="lightgray",
    width=1600,
    height=800,
    colormap="bone",
).generate(cloud_df[cloud_df["party"] == "D"]["text"][0])

# Plot the word cloud
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud_dems)
plt.axis("off")
plt.tight_layout(pad=1)
plt.show()

In [ ]:
# Generate word cloud image for Republican tweets
wordcloud_reps = WordCloud(
    stopwords=stopwords,
    background_color="lightgray",
    width=1600,
    height=800,
    colormap="gist_heat",
).generate(cloud_df[cloud_df["party"] == "R"]["text"][2])

# Plot the word cloud
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud_reps)
plt.axis("off")
plt.tight_layout(pad=1)
plt.show()


In [ ]:
# Close SQLite connection
conn.close()
